In [3]:
import json
import pandas as pd
import spacy
from collections import Counter


In [4]:
# Load Training data
with open("spider_data/spider_data/train_spider.json", "r") as f:
    train_spider_data = json.load(f)
with open("spider_data/spider_data/train_others.json", "r") as f:
    train_others_data = json.load(f)
train_data = train_spider_data + train_others_data

In [5]:
# Create Dataframes
train_data_df = pd.DataFrame(train_data)

In [6]:
train_data_df.head(2)

,db_id,query,query_toks,query_toks_no_value,question,question_toks,sql
0,department_management,SELECT count(*) FROM head WHERE age > 56,"[SELECT, count, (, *, ), FROM, head, WHERE, ag...","[select, count, (, *, ), from, head, where, ag...",How many heads of the departments are older th...,"[How, many, heads, of, the, departments, are, ...","{'from': {'table_units': [['table_unit', 1]], ..."
1,department_management,"SELECT name , born_state , age FROM head ORD...","[SELECT, name, ,, born_state, ,, age, FROM, he...","[select, name, ,, born_state, ,, age, from, he...","List the name, born state and age of the heads...","[List, the, name, ,, born, state, and, age, of...","{'from': {'table_units': [['table_unit', 1]], ..."


In [9]:
nlp = spacy.load("en_core_web_sm")

def extract_linguistic_features(question):
    doc = nlp(question)

    # general
    features = {
        "num_tokens": len(doc),
        "is_wh_question": any(token.text.lower() in ["what", "where", "who", "how", "why"] for token in doc),
        "has_negation": any(token.text.lower() in ["not", "no"] for token in doc),
        "num_entities": len(doc.ents),
        "num_unique_dependencies": len(set(token.dep_ for token in doc)),
        "dependency_tree_depth": max([token.head.i for token in doc] + [0]),
        "dependency_depth": max([len(list(token.subtree)) for token in doc]),
    }

    # POS-Tagging
    pos_counts = Counter(token.pos_ for token in doc)
    features.update({
        "num_nouns": pos_counts.get("NOUN", 0),
        "num_verbs": pos_counts.get("VERB", 0),
        "num_adjectives": pos_counts.get("ADJ", 0),
        "num_adverbs": pos_counts.get("ADV", 0),
    })

    # count entities for type
    entity_counts = Counter(ent.label_ for ent in doc.ents)
    features.update({
        "num_persons": entity_counts.get("PERSON", 0),
        "num_orgs": entity_counts.get("ORG", 0),
        "num_locations": entity_counts.get("GPE", 0),
    })

    # frequency profile (without stopwords abd interpunktion)
    word_frequencies = Counter(token.text.lower() for token in doc if not token.is_stop and not token.is_punct)
    features["frequent_words"] = sum(word_frequencies.values())
    features["rare_words"] = sum(1 for count in word_frequencies.values() if count == 1)

    return features


In [10]:
from tqdm import tqdm

# Merkmale aus den Trainingsdaten extrahieren
train_features = []
for item in tqdm(train_data):
    features = extract_linguistic_features(item["question"])
    features["question"] = item["question"]
    features["query"] = item["query"]
    train_features.append(features)

df_train = pd.DataFrame(train_features)
df_train.to_csv("output/spider_train_features.csv", index=False)

100%|██████████| 8659/8659 [00:39<00:00, 221.65it/s]


In [ ]:
df_train.head(2)

,num_tokens,num_nouns,num_verbs,num_adjectives,is_wh_question,dependency_tree_depth,average_sentence_length,num_subordinated_clauses,num_persons,num_orgs,num_locations,num_entities,num_unique_dependencies,has_negation,top_5_words,question,query
0,11,2,0,2,True,8,11.0,0,0,0,0,1,9,False,"[(heads, 1), (departments, 1)]",How many heads of the departments are older th...,SELECT count(*) FROM head WHERE age > 56
1,17,6,3,0,False,14,17.0,0,0,0,0,0,12,False,"[(age, 2), (list, 1), (name, 1), (born, 1), (s...","List the name, born state and age of the heads...","SELECT name , born_state , age FROM head ORD..."
